In [ ]:
import parcels
import xarray as xr
from datetime import timedelta, datetime
import os
import json

import random
import numpy as np
import time

import matplotlib.pyplot as plt
import pandas as pd

In [ ]:
mitgcm_results_path_arr = ['/storage/alplakes_test/ARTFUL_project/01_2023-09-12/geneva_artful_01.nc',
                           '/storage/alplakes_test/ARTFUL_project/02_2023-11-08/geneva_artful_02.nc',
                           '/storage/alplakes_test/ARTFUL_project/03_2024-05-01/geneva_artful_03.nc']

date_end_seeding_arr = [np.datetime64('2023-09-12 12:00:00'),
                        np.datetime64('2023-11-08 11:00:00'),
                        np.datetime64('2024-05-01 12:00:00')]

depth_arr = [(24,26),
             (1,26),
             (1,3)]

suffix_output = 'diff_2Cs_mixed'
output_file_path_arr = [f"02_output/artful_01_{suffix_output}.zarr",
                        f"02_output/artful_02_{suffix_output}.zarr",
                        f"02_output/artful_03_{suffix_output}.zarr"]

### Parameters

In [ ]:
i_scenario = 0
mitgcm_results_path = mitgcm_results_path_arr[i_scenario]
date_end_seeding = date_end_seeding_arr[i_scenario]

output_file_path = output_file_path_arr[i_scenario]

period_seeding = np.timedelta64(10, 'D')
freq_seeding = np.timedelta64(1, 'h')

date_start_seeding = date_end_seeding - period_seeding

In [ ]:
Cs = 0.2

n_part_per_seeding = 2000
depth_min = depth_arr[i_scenario][0] # minimum release depth
depth_max = depth_arr[i_scenario][1] # maximum release depth

(x_c,y_c) = (44275, 21375) # center of seeding point in MITgcm coordinates
r_seeding = 200 # the radius around the center in m

## Create velocity fields from MITgcm

In [ ]:
xr_mitgcm = xr.open_dataset(mitgcm_results_path)
date_ref = xr_mitgcm.time.values[0]

In [ ]:
xr_mitgcm.Z.values

In [ ]:
variables = {"U": "UVEL", "V": "VVEL", "W": "WVEL"}
dimensions = {
    "U": {
        "lon": "XG",
        "lat": "YG",
        "depth": "Z",
        "time": "time",
    },
    "V": {
        "lon": "XG",
        "lat": "YG",
        "depth": "Z",
        "time": "time",
    },
    "W": {
        "lon": "XG",
        "lat": "YG",
        "depth": "Z",
        "time": "time",
    },
}
fieldset = parcels.FieldSet.from_mitgcm(
    filenames=[mitgcm_results_path], 
    variables=variables, 
    dimensions=dimensions, 
    allow_time_extrapolation=False,
    mesh='flat') # mesh='flat' if longitudes and latitudes are given in meters, rather than degrees

In [ ]:
x = fieldset.U.grid.lon
y = fieldset.U.grid.lat

cell_areas = parcels.Field(
    name="cell_areas", data=fieldset.U.cell_areas(), lon=x, lat=y
)
fieldset.add_field(cell_areas)

fieldset.add_constant("Cs", Cs)

## Create particle set

Here we create particles randomly seeded within a defined circle.

In [ ]:
def RandomPointInCircle(R,xc,yc):
    r = R*random.random()
    theta = 2*np.pi*random.random()
    
    return xc+r*np.cos(theta),yc+r*np.sin(theta)

In [ ]:
def RandomPointInRectangle(xmin, xmax, ymin, ymax):
    return xmin+(xmax-xmin)*random.random(), ymin+(ymax-ymin)*random.random()

In [ ]:
n_part = int(n_part_per_seeding * period_seeding / freq_seeding) # total number of particles at the end
print('Total number of particles: ' + str(n_part))
print('Date ref: ' + str(date_ref))
print('Date start seeding: ' + str(date_start_seeding))
print('Date end seeding: ' + str(date_start_seeding + int(period_seeding / freq_seeding) * freq_seeding))
print('Depth min: ' + str(depth_min))
print('Depth max: ' + str(depth_max))

zi_particles = np.array([-random.uniform(depth_min, depth_max) for i in range(n_part)])
xi_particles = x_c*np.ones(n_part)
yi_particles = y_c*np.ones(n_part)

timei_particles = np.zeros(n_part)
for i in range(int(period_seeding / freq_seeding)):
    seeding_date = date_start_seeding + i * freq_seeding
    timei_particles[i * n_part_per_seeding : (i+1) * n_part_per_seeding] = (seeding_date - date_ref)/ np.timedelta64(1, 's')

for i in range(n_part):
    (xd,yd) = RandomPointInCircle(r_seeding,x_c,y_c)
    xi_particles[i] = xd
    yi_particles[i] = yd

In [ ]:
pset = parcels.ParticleSet.from_list(
    fieldset=fieldset,  # the fields on which the particles are advected
    pclass=parcels.JITParticle,  # the type of particles (JITParticle or ScipyParticle)
    lon=xi_particles,  # a vector of release longitudes
    lat=yi_particles,  # a vector of release latitudes
    depth=zi_particles,
    time=timei_particles
)

In [ ]:
plt.figure(figsize=(15,7))
plt.pcolormesh(fieldset.U.grid.lon, fieldset.U.grid.lat, xr_mitgcm['THETA'].isel(Z=0, time= -1).values)
plt.xlabel("X [m]")
plt.ylabel("Y [m]")
plt.colorbar()

plt.scatter(pset.lon[0:n_part_per_seeding], pset.lat[0:n_part_per_seeding], s=3, c='r')
#plt.xlim(left=35000, right=50000)
#plt.ylim(bottom=15000)
plt.show()

## Create a new kernel for Smagorinsky diffusion method

https://docs.oceanparcels.org/en/latest/examples/tutorial_diffusion.html

In [ ]:
def smagdiff(particle, fieldset, time):
    dx = 50
    # gradients are computed by using a local central difference.
    updx, vpdx = fieldset.UV[time, particle.depth, particle.lat, particle.lon + dx]
    umdx, vmdx = fieldset.UV[time, particle.depth, particle.lat, particle.lon - dx]
    updy, vpdy = fieldset.UV[time, particle.depth, particle.lat + dx, particle.lon]
    umdy, vmdy = fieldset.UV[time, particle.depth, particle.lat - dx, particle.lon]

    dudx = (updx - umdx) / (2 * dx)
    dudy = (updy - umdy) / (2 * dx)

    dvdx = (vpdx - vmdx) / (2 * dx)
    dvdy = (vpdy - vmdy) / (2 * dx)

    A = fieldset.cell_areas[time, 0, particle.lat, particle.lon]
    Kh = fieldset.Cs * A * math.sqrt(dudx**2 + 0.5 * (dudy + dvdx) ** 2 + dvdy**2)

    dlat = parcels.ParcelsRandom.normalvariate(0.0, 1.0) * math.sqrt(
        2 * math.fabs(particle.dt) * Kh
    )
    dlon = parcels.ParcelsRandom.normalvariate(0.0, 1.0) * math.sqrt(
        2 * math.fabs(particle.dt) * Kh
    )

    particle_dlat += dlat
    particle_dlon += dlon

## Set output parameters

In [ ]:
output_file = pset.ParticleFile(
    name=output_file_path,  # the file name
    outputdt=timedelta(hours=1),  # the time step of the outputs
)

In [ ]:
period_seeding.astype('timedelta64[D]').astype(int)

## Run simulation

In [ ]:
start_clock = time.time()
pset.execute(
    [parcels.AdvectionRK4, smagdiff],  # the kernel (which defines how particles move)
    runtime=timedelta(days=int(period_seeding.astype('timedelta64[D]').astype(int)), hours=4),  # the total length of the run
    dt=timedelta(minutes=1),  # the timestep of the kernel
    output_file=output_file,
)
end_clock = time.time()
print(f'Done. Operation took {end_clock - start_clock}s.')